In [ ]:
import numpy as np


class NeuralNetwork(object):
    
    def __init__(self):
        self.input_layer_size = 2
        self.hidden_layer_size = 3
        self.output_layer_size = 1
        
        self.w1 = np.random.randn(
            self.input_layer_size,
            self.hidden_layer_size)
        self.w2 = np.random.randn(
            self.hidden_layer_size,
            self.output_layer_size)
        
    def forward(self, x):
        """ Propagate inputs through network. """
        self.z2 = np.dot(x, self.w1)
        self.a2 = self.sigmod(self.z2)
        self.z3 = np.dot(self.a2, self.w2)
        yhat = self.sigmoid(self.z3)
        return yhat
    
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
        